In [67]:
from dash import dcc, html, Input, Output, Dash, callback
import plotly.express as px
import pandas as pd
import numpy as np

In [28]:
data = pd.read_csv(
    "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"
)
data.drop(columns=["Unnamed: 0"], inplace=True)

In [290]:
app = Dash(title="SpaceX Dashboard")

In [291]:
sites = []
for x in data["Launch Site"].unique():
    sites.append(x)

In [292]:
app.layout = html.Div(
    [
        html.Div(html.H1("SpaceX Dashboard", style={"textAlign": "center"})),
        html.Hr(),
        dcc.Dropdown(
            id="site-dropdown",
            options=[{"label": "All Sites", "value": "ALL"}]
            + [
                {"label": x, "value": x} for x in sites
            ],  # Dict comprehension for adding all sites in the dropdown
            value="ALL",
            placeholder="Select a launch site here",
            searchable=True,
        ),
        dcc.Graph(id="success-pie-chart"),
        html.Hr(),
        dcc.RangeSlider(
            id="payload-slider",
            min=0,
            max=10000,
            step=1000,
            value=[0, 10000],
        ),
        dcc.Graph(id="success-payload-scatter-chart"),
    ]
)  # End main div

In [293]:
@app.callback(
    Output(component_id="success-pie-chart", component_property="figure"),
    Input(component_id="site-dropdown", component_property="value"),
)
def get_pie_chart(selection):
    if selection == "ALL":
        fig = px.pie(
            data_frame=data,
            names="Launch Site",
            values="class",
            title="Total successful launch rate by site",
        ).update_traces(marker_line_color="black", marker_line_width=1)
    else:
        site_success = (
            data[data["Launch Site"] == selection]
            .groupby(["class"], as_index=False)["class"]
            .value_counts()
        )
        fig = px.pie(
            data_frame=site_success,
            names=site_success["class"],
            values=site_success["count"],
            title = f'Successful launch rate for site {selection}'
        ).update_traces(marker_line_color="black", marker_line_width=1)

    return fig

In [294]:
@app.callback(
    Output(component_id="success-payload-scatter-chart", component_property="figure"),
    Input(component_id="site-dropdown", component_property="value"),
    Input(component_id="payload-slider", component_property="value"),
)
def get_scatter_plot(selection, range):

    if selection == "ALL":
        # data_filtered = data[
        #     (data["Payload Mass (kg)"] >= range[0])
        #     & (data["Payload Mass (kg)"] <= range[1])
        # ]
        fig = px.scatter(
            data_frame=data,
            x="Payload Mass (kg)",
            y="class",
            color="Booster Version Category",
            title="Correlation between payload and success for all sites",
        )
        return fig

    elif selection != "ALL":
        data_filtered1 = data[data["Launch Site"] == selection]
        
        data_filtered2 = data_filtered1[
            (data_filtered1["Payload Mass (kg)"] >= range[0])
            & (data_filtered1["Payload Mass (kg)"] <= range[1])
        ]
        fig = px.scatter(
            data_frame=data_filtered2,
            x="Payload Mass (kg)",
            y="class",
            color="Booster Version Category",
            title=f"Correlation between payload and success for site {selection}",
        )
        return fig

In [295]:
app.run(jupyter_mode="external")

Dash app running on http://127.0.0.1:8050/
